In [ ]:
# === Google Drive 마운트 ===
from google.colab import drive
drive.mount('/content/drive')

# === 경로 설정 ===
MERGED_MODEL_PATH =   "/content/drive/MyDrive/data/test2/experiments_lora/r16_a32_do0.05_lr0.0003/sft_tuned_model_epoch_4"

# 출력 GGUF 파일(Colab 로컬)
GGUF_OUT_PATH = "/content/exaone35-2.4b-Q4_K_M.gguf"

# GGUF 백업 파일(Drive)
GGUF_BACKUP_PATH = "/content/drive/MyDrive/data/test2/exaone35-2.4b-Q4_K_M.gguf"
GGUF_F16 = "/content/exaone35-2.4b-f16.gguf"
print("MERGED_MODEL_PATH:", MERGED_MODEL_PATH)
print("GGUF_OUT_PATH    :", GGUF_OUT_PATH)
print("GGUF_BACKUP_PATH :", GGUF_BACKUP_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MERGED_MODEL_PATH: /content/drive/MyDrive/data/test2/experiments_lora/r16_a32_do0.05_lr0.0003/sft_tuned_model_epoch_4
GGUF_OUT_PATH    : /content/exaone35-2.4b-Q4_K_M.gguf
GGUF_BACKUP_PATH : /content/drive/MyDrive/data/test2/exaone35-2.4b-Q4_K_M.gguf


In [ ]:
import torch
print("PyTorch:", torch.__version__)
if torch.cuda.is_available():
    print("CUDA:", torch.version.cuda, "| GPU:", torch.cuda.get_device_name(0))

PyTorch: 2.8.0+cu126
CUDA: 12.6 | GPU: NVIDIA A100-SXM4-40GB


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print(f"'{MERGED_MODEL_PATH}'에서 최종 병합 모델을 로드합니다...")

final_model = AutoModelForCausalLM.from_pretrained(
    MERGED_MODEL_PATH,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
final_tokenizer = AutoTokenizer.from_pretrained(MERGED_MODEL_PATH, trust_remote_code=True)

# 패딩 설정(경고 방지)
final_tokenizer.padding_side = "left"
if final_tokenizer.pad_token is None:
    final_tokenizer.pad_token = final_tokenizer.eos_token

# (필요 시) lm_head tie 워크어라운드
try:
    if hasattr(final_model, "get_output_embeddings") and hasattr(final_model, "lm_head"):
        final_model.lm_head.weight = final_model.get_output_embeddings().weight
        print("✅ lm_head.weight tied to output embeddings.")
except Exception as e:
    print("lm_head tie skip:", e)

# 간단 생성 테스트
prompt = "오늘은 어떤 날이냐?"
inputs = final_tokenizer(prompt, return_tensors="pt").to(final_model.device)
outputs = final_model.generate(**inputs, max_new_tokens=64, do_sample=False)
print(final_tokenizer.decode(outputs[0], skip_special_tokens=True))


'/content/drive/MyDrive/data/test2/experiments_lora/r16_a32_do0.05_lr0.0003/sft_tuned_model_epoch_4'에서 최종 병합 모델을 로드합니다...
✅ lm_head.weight tied to output embeddings.
오늘은 어떤 날이냐? 오늘은 2013년 10월 26일, 제20대 국회에서 '국가인권위원회법'이 제정되었던 날이었노라. 이는 우리 사회의 인권 수준을 한 단계 높이는 중요한 발걸음이었느니라. 이 법의 제정


In [ ]:
# llama.cpp 가져오기 + 변환 스크립트 의존성
!git clone https://github.com/ggml-org/llama.cpp
!pip -q install -r llama.cpp/requirements.txt
print("✅ llama.cpp 및 변환 의존성 준비 완료")


Cloning into 'llama.cpp'...
remote: Enumerating objects: 64532, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 64532 (delta 127), reused 64 (delta 53), pack-reused 64312 (from 3)
Receiving objects: 100% (64532/64532), 172.86 MiB | 40.37 MiB/s, done.
Resolving deltas: 100% (46841/46841), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 113.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py \
  --outfile "{GGUF_F16}" \
  --outtype f16 \
  "{MERGED_MODEL_PATH}"

import os
print("f16 GGUF 생성 여부:", os.path.exists(GGUF_F16), "->", GGUF_F16)

INFO:hf-to-gguf:Loading model: sft_tuned_model_epoch_4
 You can inspect the repository content at https://hf.co//content/drive/MyDrive/data/test2/experiments_lora/r16_a32_do0.05_lr0.0003/sft_tuned_model_epoch_4.
Please pass the argument `trust_remote_code=True` to allow custom code to be run.
INFO:hf-to-gguf:Model architecture: ExaoneForCausalLM
 You can inspect the repository content at https://hf.co//content/drive/MyDrive/data/test2/experiments_lora/r16_a32_do0.05_lr0.0003/sft_tuned_model_epoch_4.
Please pass the argument `trust_remote_code=True` to allow custom code to be run.
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {40}
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {2560, 640}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> F16, shape = {25

In [ ]:
%cd /content/llama.cpp

!cmake -S . -B build -DLLAMA_NATIVE=ON
!cmake --build build -j
!ls -al build/bin

/content/llama.cpp
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
CMake Warning at CMakeLists.txt:126 (message):
  LLAMA_NATIVE is deprecated and will be removed in the future.

  Use GGML_NATIVE instead

Call Stack (most recent call first):
  CMakeLists.txt:135 (llama_option_depr)


-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found T

In [ ]:
import os, glob, subprocess

GGUF_F16 = "/content/exaone35-2.4b-f16.gguf"
GGUF_Q4  = "/content/exaone35-2.4b-Q4_K_M.gguf"

# quantize 실행파일 자동 탐색
quant_bins = glob.glob("/content/llama.cpp/**/llama-quantize", recursive=True)
print("🔍 Found quantize binaries:", quant_bins)

if not quant_bins:
    raise FileNotFoundError("quantize 실행 파일을 찾지 못했습니다. build/bin 또는 bin/ 아래를 확인하세요.")

quant_path = quant_bins[0]
print(f"✅ Using quantize: {quant_path}")

# 실행 권한 부여 후 양자화 실행
os.system(f"chmod +x '{quant_path}'")
os.system(f"'{quant_path}' '{GGUF_F16}' '{GGUF_Q4}' q4_K_M")

print("\n=== 결과 확인 ===")
print("Q4_K_M GGUF 생성:", os.path.exists(GGUF_Q4), "->", GGUF_Q4)


🔍 Found quantize binaries: ['/content/llama.cpp/build/bin/llama-quantize']
✅ Using quantize: /content/llama.cpp/build/bin/llama-quantize

=== 결과 확인 ===
Q4_K_M GGUF 생성: True -> /content/exaone35-2.4b-Q4_K_M.gguf


In [ ]:
GGUF_F16 = "/content/exaone35-2.4b-f16.gguf"
GGUF_Q4  = "/content/exaone35-2.4b-Q4_K_M.gguf"
QUANT    = "/content/llama.cpp/build/bin/llama-quantize" 

!chmod +x "{QUANT}"
!"{QUANT}" "{GGUF_F16}" "{GGUF_Q4}" q4_K_M

import os
print("Q4_K_M GGUF 생성:", os.path.exists(GGUF_Q4), "->", GGUF_Q4)


main: build = 6727 (cdb6da46)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/exaone35-2.4b-f16.gguf' to '/content/exaone35-2.4b-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 25 key-value pairs and 273 tensors from /content/exaone35-2.4b-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = exaone
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Sft_Tuned_Model_Epoch_4
llama_model_loader: - kv   3:                         general.size_label str              = 2.4B
llama_model_loader: - kv   4:                    exaone.embedding_length u32              = 2560
llama_model_loader: - kv   5:             

In [ ]:
import glob, os

GGUF_Q4 = "/content/exaone35-2.4b-Q4_K_M.gguf"

# llama-cli 위치 자동 탐색 (build/bin 또는 bin 등)
cli_candidates = glob.glob("/content/llama.cpp/**/llama-cli", recursive=True)
print("Found llama-cli:", cli_candidates)
assert cli_candidates, "llama-cli 실행파일을 찾지 못했습니다. 빌드가 완료되었는지 확인하세요."

CLI = cli_candidates[0]
!chmod +x "{CLI}"

prompt = "시스템: 너는 세종대왕 말투로, 간결하고 품위 있게 대답한다.\n사용자: 오늘 할 일을 세 줄로 정리해주시오.\n어시스턴트:"

!"{CLI}" -m "{GGUF_Q4}" \
  -p "{prompt}" \
  -n 128 -c 4096 --temp 0.7 --top-p 0.9 \
  -r "### 응답:" -r "\n사용자:" -r "\n시스템:" -r "\n어시스턴트:"


Found llama-cli: ['/content/llama.cpp/build/bin/llama-cli']
build: 6727 (cdb6da46) with cc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0 for x86_64-linux-gnu
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 25 key-value pairs and 273 tensors from /content/exaone35-2.4b-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = exaone
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Sft_Tuned_Model_Epoch_4
llama_model_loader: - kv   3:                         general.size_label str              = 2.4B
llama_model_loader: - kv   4:                    exaone.embedding_length u32              = 2560
llama_model_loader: - 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import shutil, os
GGUF_Q4      = "/content/exaone35-2.4b-Q4_K_M.gguf"
GGUF_BACKUP  = "/content/drive/MyDrive/exaone35-2.4b-Q4_K_M.gguf"

if os.path.exists(GGUF_Q4):
    shutil.copy(GGUF_Q4, GGUF_BACKUP)
    print("✅ Drive 백업 완료:", GGUF_BACKUP)
else:
    print("❌ GGUF가 없습니다. 양자화 단계가 성공했는지 확인하세요.")
